<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [1]:
import numpy as np
import pandas as pd

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
aux = np.core.defchararray.split(corpus, sep = ' ') # transformo los documentos en listas de términos
aux = np.concatenate(aux)                           # concateno todos los términos en una sola lista
vocab = np.unique(aux)                              # obtengo el vocabulario de términos sin repetir
vocab

array(['de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas',
       'que'], dtype='<U7')

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [5]:
aux = np.core.defchararray.split(corpus[0], sep = ' ')
aux

array(list(['que', 'dia', 'es', 'hoy']), dtype=object)

In [6]:
one_hot = pd.DataFrame(columns = vocab)

In [7]:
# Defino función que entrega el vector de one hot encoding del texto pasado como argumento en función del vocabulario.

def get_one_hot_vector(text, vocab, verbosity=0):
  one_hot_vector = []
  for i in vocab:
    if i in text.split(): 
      one_hot_vector.append(1)
    else: 
      one_hot_vector.append(0)
  if verbosity: print('One hot vector for \''+text+'\': '+str(one_hot_vector))
  return one_hot_vector

In [8]:
test = get_one_hot_vector(corpus[0], vocab, verbosity=True)

One hot vector for 'que dia es hoy': [0, 1, 0, 1, 0, 1, 0, 0, 1]


In [9]:
for i in corpus:
  one_hot.loc[len(one_hot.index)] = get_one_hot_vector(i, vocab, True)
one_hot

One hot vector for 'que dia es hoy': [0, 1, 0, 1, 0, 1, 0, 0, 1]
One hot vector for 'martes el dia de hoy es martes': [1, 1, 1, 1, 0, 1, 1, 0, 0]
One hot vector for 'martes muchas gracias': [0, 0, 0, 0, 1, 0, 1, 1, 0]


,de,dia,el,es,gracias,hoy,martes,muchas,que
0,0,1,0,1,0,1,0,0,1
1,1,1,1,1,0,1,1,0,0
2,0,0,0,0,1,0,1,1,0


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [10]:
frec_matrix = pd.DataFrame(columns = vocab)

In [11]:
def get_frec_vector(text, vocab, verbosity=0):
  frec_vector = []
  for i in vocab:
    frec_vector.append(text.split().count(i))
  if verbosity: print('Frec vector for \''+text+'\': '+str(frec_vector))
  return frec_vector

In [12]:
get_frec_vector(corpus[1], vocab, verbosity=True)

Frec vector for 'martes el dia de hoy es martes': [1, 1, 1, 1, 0, 1, 2, 0, 0]


[1, 1, 1, 1, 0, 1, 2, 0, 0]

In [13]:
for i in corpus:
  frec_matrix.loc[len(frec_matrix.index)] = get_frec_vector(i, vocab, True)
frec_matrix

Frec vector for 'que dia es hoy': [0, 1, 0, 1, 0, 1, 0, 0, 1]
Frec vector for 'martes el dia de hoy es martes': [1, 1, 1, 1, 0, 1, 2, 0, 0]
Frec vector for 'martes muchas gracias': [0, 0, 0, 0, 1, 0, 1, 1, 0]


,de,dia,el,es,gracias,hoy,martes,muchas,que
0,0,1,0,1,0,1,0,0,1
1,1,1,1,1,0,1,2,0,0
2,0,0,0,0,1,0,1,1,0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [14]:
import math
# IDF
idf_vector = []

for i in vocab:
  count = 0
  for j in corpus:
    if i in j.split(): 
      count=count+1
  idf_vector.append(math.log10(len(corpus)/count))


In [15]:
idf_vector

[0.47712125471966244,
 0.17609125905568124,
 0.47712125471966244,
 0.17609125905568124,
 0.47712125471966244,
 0.17609125905568124,
 0.17609125905568124,
 0.47712125471966244,
 0.47712125471966244]

In [16]:
TF_IDF = pd.DataFrame(frec_matrix.values * idf_vector, columns = vocab)
TF_IDF

,de,dia,el,es,gracias,hoy,martes,muchas,que
0,0.0,0.176091,0.0,0.176091,0.0,0.176091,0.0,0.0,0.477121
1,0.477121,0.176091,0.477121,0.176091,0.0,0.176091,0.352183,0.0,0.0
2,0.0,0.0,0.0,0.0,0.477121,0.0,0.176091,0.477121,0.0


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [78]:
import scipy.stats as ss

def cosine_order(corpus, vocab, doc_index):
  one_hot_matrix = []
  for i in corpus:
    one_hot_matrix.append(get_one_hot_vector(i, vocab))
  
  cosine_vector = []
  for i in one_hot_matrix:
    cosine_vector.append(cosine_similarity(one_hot_matrix[doc_index], i))
  aux_pd = pd.DataFrame(corpus, index = ss.rankdata(cosine_vector))
  
  return aux_pd.sort_index(ascending = False)

In [81]:
cosine_order(corpus, vocab, 0)

,0
3.0,que dia es hoy
2.0,martes el dia de hoy es martes
1.0,martes muchas gracias
